- Purpose:
    - create a transcript id to gene id two-column map with no headers that can be used for salmon
    - create a table with trascript id gene id and gene name useful for DEG and other downstream analyses

In [1]:
#####################
# import statements #
#####################
import pysam
import pandas as pd
from tqdm import tqdm
import os
import gzip
from Bio import SeqIO

In [ ]:
##########################
# User-Defined Variables #
##########################
# - define all variables below with paths to the required files
# - this should be the only cell that requires modification
# - both files should be compressed with bgzip and indexed

gtfgz_path = ''
transcript_fa_path = ''

In [3]:
# set output directory based on gtf path
out_dir = os.path.split(gtfgz_path)[0]

# read gtf and print the first 20 entries
gtfgz = pysam.TabixFile(gtfgz_path)

count = 0
for gtf_line in gtfgz.fetch():
    print(gtf_line)
    if (count := count + 1) > 19:
        break

gtf_feature_count = 0
for gtf_line in gtfgz.fetch():
    gtf_feature_count += 1

chr1	HAVANA	exon	11121	11211	.	+	.	gene_id "ENSG00000290825.2"; transcript_id "ENST00000832824.1"; gene_type "lncRNA"; gene_name "DDX11L16"; transcript_type "lncRNA"; transcript_name "DDX11L16-260"; exon_number 1; exon_id "ENSE00004248723.1"; level 2; tag "TAGENE";
chr1	HAVANA	gene	11121	24894	.	+	.	gene_id "ENSG00000290825.2"; gene_type "lncRNA"; gene_name "DDX11L16"; level 2; tag "overlaps_pseudogene";
chr1	HAVANA	transcript	11121	14413	.	+	.	gene_id "ENSG00000290825.2"; transcript_id "ENST00000832824.1"; gene_type "lncRNA"; gene_name "DDX11L16"; transcript_type "lncRNA"; transcript_name "DDX11L16-260"; level 2; tag "TAGENE";
chr1	HAVANA	exon	11125	11211	.	+	.	gene_id "ENSG00000290825.2"; transcript_id "ENST00000832825.1"; gene_type "lncRNA"; gene_name "DDX11L16"; transcript_type "lncRNA"; transcript_name "DDX11L16-261"; exon_number 1; exon_id "ENSE00004248721.1"; level 2; tag "TAGENE";
chr1	HAVANA	transcript	11125	14405	.	+	.	gene_id "ENSG00000290825.2"; transcript_id "ENST000008328

In [4]:
# from the gtf, compile matched lists of gene ids, gene names, and gene types for all gene features
gene_list = []
gene_name_list = []
gene_type_list = []

for gtf_line in tqdm(gtfgz.fetch(),total=gtf_feature_count):
    gtf_rec = gtf_line.strip().split('\t')
    if gtf_rec[2] == 'gene':
        gene_id = gtf_rec[8].strip().split('gene_id "')[1].strip().split('"')[0].strip()
        gene_name = gtf_rec[8].strip().split('gene_name "')[1].strip().split('"')[0].strip()
        gene_type = gtf_rec[8].strip().split('gene_type "')[1].strip().split('"')[0].strip() if 'gene_type' in gtf_rec[8] else ''

        gene_list.append(gene_id)
        gene_name_list.append(gene_name)
        gene_type_list.append(gene_type)

# compile lists into a dataframe
gene_map_df = pd.DataFrame({
    'gene_id':gene_list,
    'gene_name':gene_name_list,
    'gene_type':gene_type_list,
})
display(gene_map_df)

100%|██████████| 4119244/4119244 [00:03<00:00, 1092779.69it/s]


,gene_id,gene_name,gene_type
0,ENSG00000290825.2,DDX11L16,lncRNA
1,ENSG00000223972.6,DDX11L1,transcribed_unprocessed_pseudogene
2,ENSG00000310526.1,WASH7P,lncRNA
3,ENSG00000227232.6,WASH7P,transcribed_unprocessed_pseudogene
4,ENSG00000278267.1,MIR6859-1,miRNA
...,...,...,...
78889,ENSG00000303867.1,ENSG00000303867,lncRNA
78890,ENSG00000303902.1,ENSG00000303902,lncRNA
78891,ENSG00000306528.1,ENSG00000306528,lncRNA
78892,ENSG00000297844.1,ENSG00000297844,lncRNA


In [5]:
# from the gtf, compile matched lists of transcript ids, gene ids, and gene names for all transcript features
transcript_list = []
gene_list = []
gene_name_list = []

for gtf_line in tqdm(gtfgz.fetch(),total=gtf_feature_count):
    gtf_rec = gtf_line.strip().split('\t')
    if gtf_rec[2] == 'transcript':
        transcript_list.append(gtf_rec[8].strip().split('transcript_id "')[1].strip().split('"')[0].strip())
        gene_list.append(gtf_rec[8].strip().split('gene_id "')[1].strip().split('"')[0].strip())
        gene_name_list.append(gtf_rec[8].strip().split('gene_name "')[1].strip().split('"')[0].strip())

# compile the transcript ids and gene ids into map_df and write an otput tsv of two columns with no headers that maps transcript_id to gene_id
# used as a salmon input
map_df = pd.DataFrame({
    'transcript_id':transcript_list,
    'gene_id':gene_list,
})
map_tsv_path = os.path.join(out_dir, 'transcript-gene-map.tsv')
map_df.to_csv(map_tsv_path, index=False, header=False, sep='\t')

# add gene names as a third column to the df
map_df['gene_name'] = gene_name_list
display(map_df)

100%|██████████| 4119244/4119244 [00:04<00:00, 912631.82it/s]


,transcript_id,gene_id,gene_name
0,ENST00000832824.1,ENSG00000290825.2,DDX11L16
1,ENST00000832825.1,ENSG00000290825.2,DDX11L16
2,ENST00000832826.1,ENSG00000290825.2,DDX11L16
3,ENST00000832827.1,ENSG00000290825.2,DDX11L16
4,ENST00000832828.1,ENSG00000290825.2,DDX11L16
...,...,...,...
387949,ENST00000797723.1,ENSG00000303867.1,ENSG00000303867
387950,ENST00000819228.1,ENSG00000306528.1,ENSG00000306528
387951,ENST00000819229.1,ENSG00000306528.1,ENSG00000306528
387952,ENST00000751368.1,ENSG00000297844.1,ENSG00000297844


In [6]:
# from the transcripts fasta, compile matched lists of transcript ids, gene ids, and gene names for all gene transcript features
transcript_contigs_transcript_list = []
transcript_contigs_gene_list = []
transcript_contigs_gene_name_list = []

with gzip.open(transcript_fa_path, 'rt') as fasta:
    for fa_rec in SeqIO.parse(fasta, 'fasta'):
        transcript_contigs_rec = fa_rec.id.strip().split('|')
        transcript_contigs_transcript_list.append(transcript_contigs_rec[0].strip())
        transcript_contigs_gene_list.append(transcript_contigs_rec[1].strip())
        transcript_contigs_gene_name_list.append(transcript_contigs_rec[5].strip())

# compile into dataframe
transcript_contigs_df = pd.DataFrame({
    'transcript_id':transcript_contigs_transcript_list,
    'gene_id':transcript_contigs_gene_list,
    'gene_name':transcript_contigs_gene_name_list,
})
display(transcript_contigs_df)

,transcript_id,gene_id,gene_name
0,ENST00000832824.1,ENSG00000290825.2,DDX11L16
1,ENST00000832825.1,ENSG00000290825.2,DDX11L16
2,ENST00000832826.1,ENSG00000290825.2,DDX11L16
3,ENST00000832827.1,ENSG00000290825.2,DDX11L16
4,ENST00000832828.1,ENSG00000290825.2,DDX11L16
...,...,...,...
387949,ENST00000797959.1,ENSG00000303902.1,ENSG00000303902
387950,ENST00000819228.1,ENSG00000306528.1,ENSG00000306528
387951,ENST00000819229.1,ENSG00000306528.1,ENSG00000306528
387952,ENST00000751368.1,ENSG00000297844.1,ENSG00000297844


In [7]:
# verify that gtf and fasta transcripts match, ideally in the output all three counts will be the same and the unique entries dataframes will be empty
print(f'Transcripts from gtf:          \t{len(map_df)}')
print(f'Transcripts from transcript fa:\t{len(transcript_contigs_df)}')
print(f'Transcripts in both:           \t{len(pd.merge(left=map_df,right=transcript_contigs_df,on=['transcript_id'],how='inner'))}')
print(f'Entries unique to transcript fa:\n')
unique_to_fa_df = transcript_contigs_df[~(transcript_contigs_df['transcript_id'].isin(map_df['transcript_id']))]
display(unique_to_fa_df)
print(f'Entries unique to gtf:\n')
unique_to_gtf_df = map_df[~(map_df['transcript_id'].isin(transcript_contigs_df['transcript_id']))]
display(unique_to_gtf_df)

Transcripts from gtf:          	387954
Transcripts from transcript fa:	387954
Transcripts in both:           	387954
Entries unique to transcript fa:



,transcript_id,gene_id,gene_name


Entries unique to gtf:



,transcript_id,gene_id,gene_name


In [8]:
map_tsv_path = os.path.join(out_dir, 'transcript-gene-map-w-names.tsv')
map_df.to_csv(map_tsv_path, index=False, sep='\t')  